In [1]:
# load libraries
%matplotlib inline
import pandas as pd
import numpy as np
import time
import os
import statistics as stats
import math
from scipy import stats as tests

In [2]:
#see how many rPPG files there are
files = [f for f in os.listdir('rppg_csvs') if not f.startswith('.')]
num_rppgs = len(files)

#load in all of the rPPG data into a single list of dataframes
rppg_data = []
for i in range(num_rppgs):
    userID = os.path.splitext(files[i])[0][:5]
    user_data = pd.read_csv("rppg_csvs/"+userID+".mp4-heart-rate.csv", header=0)
    rppg_data.append((userID,user_data))

In [3]:
#see how many subject files there are
files = [f for f in os.listdir('subject_csvs_clean') if not f.startswith('.')]
num_subjs = len(files)

#load in all of the subject data into a single list of dataframes
affect_data = []
for i in range(num_subjs):
    userID = os.path.splitext(files[i])[0]
    user_data = pd.read_csv("subject_csvs_clean/"+userID+".csv", header=0)
    affect_data.append((userID,user_data))

In [4]:
#Read in the timestamp and demographic data from the database
df_timestamps = pd.read_csv("timestamps.csv", header=0)
df_timestamps.head()

,userID,prolificID,name,date,vid_consent,starttime,age,gender,wherelive,mood,...,vote,likeJoe,likeTrump,abortion,guncontrol,healthcare,policing,studyabout,finishtime,comment
0,2clJY,5c19a630257db80001a2b3a8,paul benevich,11/24/20,canUseVid,10:11:26 PM,65,Male,Yes,4,...,Trump,1,5,ProLife,AntiGunControl,AntiHealthcare,ProPolice,politics,10:31:41 PM,NaN
1,ifh7Z,5e8790801eb5090afd11e178,Caitlin Sisk,11/24/20,canUseVid,10:18:00 PM,26,Female,Yes,2,...,Biden,3,2,ProLife,ProGunControl,AntiHealthcare,ProPolice,How emotional reactions to politicians voicing...,10:38:37 PM,NaN
2,1XVI6,5ed70903f3b2f8195f919e6f,Charles Edward Geyer,11/24/20,canUseVid,10:21:06 PM,35,Male,Yes,4,...,Trump,2,4,ProLife,AntiGunControl,AntiHealthcare,ProPolice,What your reactions are to policies forwarded ...,10:42:36 PM,NaN
3,PPckh,5f4926497bf47b4392f9b163,Sara Owen,25-Nov-20,canUseVid,4:15:02 PM,21,Other,Yes,4,...,Biden,3,1,ProChoice,AntiGunControl,ProHealthcare,AntiPolice,How people&#39;s reactions change when they ar...,4:33:48 PM,NaN
4,yFcTy,5faef59ae8f4f50f9ab78386,Nathan Yousman,11/25/20,canUseVid,4:56:21 PM,19,Male,Yes,3,...,Biden,3,1,ProChoice,ProGunControl,ProHealthcare,AntiPolice,Showing people videos of the two major candida...,5:15:40 PM,NaN


In [5]:
# IFF we get a negative result, we need to set noon and midnight equal to 0:00
# while avoiding doing that if it tips form 11:00 to 12:00
def clocks_are_hard(time):
    if time < 0:
        return time + (3600*12)
    else:
        return time

In [6]:
#Get all of the video timestimes and put them in the same order
def retreive_vid_times(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    time_array = times.at[times.index[0],"starttime"].split()[0].split(':')
    start_time = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])
    
    vid_times = [[0,0],[0,0],[0,0],[0,0]]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[0][0] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[0][1] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
        elif vid_order[i] == '1':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[1][0] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[1][1] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
        elif vid_order[i] == '2':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[2][0] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[2][1] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
        elif vid_order[i] == '3':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[3][0] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[3][1] = clocks_are_hard((3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2]))- start_time)

    return vid_times

In [7]:
print(retreive_vid_times(14))

[[291.0, 493.0], [532.0, 699.0], [741.0, 924.0], [79.0, 244.0]]


In [8]:
def retreive_affiliation(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    responses = df_timestamps[df_timestamps["userID"] == userID]
    party = responses.at[responses.index[0],'party']
    ideology = responses.at[responses.index[0],'ideology']
    candidate = responses.at[responses.index[0],'vote']
    like_joe = responses.at[responses.index[0],'likeJoe']
    like_trump = responses.at[responses.index[0],'likeTrump']

    return [userID, party,ideology,candidate,like_joe,like_trump]

In [9]:
print(retreive_affiliation(0))

['j7WOU', 'Republican', 'Right', 'Trump', 3, 4]


In [10]:
#Create a master list of which timestamps correspond to which videos for each user
compiled_vid_times = []
for i in range(num_subjs):
    compiled_vid_times.append(retreive_vid_times(i))
compiled_vid_times

[[[357.0, 562.0], [849.0, 1025.0], [612.0, 795.0], [107.0, 303.0]],
 [[803.0, 1082.0], [305.0, 490.0], [551.0, 764.0], [66.0, 262.0]],
 [[833.0, 1053.0], [369.0, 549.0], [594.0, 779.0], [141.0, 326.0]],
 [[748.0, 948.0], [541.0, 705.0], [319.0, 500.0], [112.0, 279.0]],
 [[855.0, 1070.0], [382.0, 560.0], [601.0, 802.0], [149.0, 338.0]],
 [[454.0, 674.0], [964.0, 1167.0], [222.0, 403.0], [721.0, 917.0]],
 [[1196.0, 1462.0], [1518.0, 1714.0], [1751.0, 1978.0], [972.0, 1153.0]],
 [[935.0, 1133.0], [259.0, 448.0], [709.0, 897.0], [499.0, 668.0]],
 [[909.0, 1128.0], [71.0, 284.0], [353.0, 580.0], [642.0, 853.0]],
 [[982.0, 1056.0], [118.0, 345.0], [711.0, 918.0], [434.0, 649.0]],
 [[632.0, 896.0], [946.0, 1156.0], [359.0, 576.0], [79.0, 282.0]],
 [[314.0, 521.0], [559.0, 743.0], [797.0, 988.0], [77.0, 265.0]],
 [[443.0, 679.0], [974.0, 1174.0], [194.0, 394.0], [730.0, 930.0]],
 [[372.0, 602.0], [116.0, 311.0], [936.0, 1131.0], [678.0, 869.0]],
 [[291.0, 493.0], [532.0, 699.0], [741.0, 924.0]

In [11]:
sentiment_df = pd.read_csv("text_sentiment.csv", header=0)
sentiment_df.head()

,userID,vid1_pol,vid1_subj,vid2_pol,vid2_subj,vid3_pol,vid3_subj,vid4_pol,vid4_subj
0,1XVI6,0.1125,0.5984,-0.0417,0.5542,0.1789,0.4378,-0.0654,0.6196
1,5yMwL,-0.0833,0.3333,0.2000,0.5000,-0.8000,0.8000,-0.5750,1.0000
2,GFPux,0.2469,0.4031,0.0750,0.3417,-0.0357,0.5929,0.0667,0.5264
3,mOhA6,0.4700,0.5800,0.4333,0.4667,0.3917,0.5083,-0.0500,0.5917
4,mTe10,-0.1500,0.6444,0.2600,0.6500,0.7000,0.6000,0.0000,0.0000


In [12]:
#compute summary statistics for each video for each participant
def compute_summary_stats(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    vid_times = compiled_vid_times[index]
    
    for r in rppg_data:
        if r[0] == userID:
            rppg_df = r[1]
            break
            
    disgust = []
    joy = []
    engagement = []
    valence = []
    rppg = []
    
    print(userID)  #progress tracker

    #compute the average affect in each facial affect category during the relevant times
    for i in range(4):
        time1 = vid_times[i][0]
        time2 = vid_times[i][1]
        rows = affect_df['time_stamp'].between(time1, time2)
        temp1 = []
        temp2 = []
        temp3 = []
        temp4 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(affect_df.at[j,'disgust'])
                temp2.append(affect_df.at[j,'joy'])
                temp3.append(affect_df.at[j,'engagement'])
                temp4.append(affect_df.at[j,'valence'])
        #disgust.append(stats.mean(temp1))
        #joy.append(stats.mean(temp2))
        #engagement.append(stats.mean(temp3))
        #valence.append(stats.mean(temp4))
        disgust.append(max(temp1))
        joy.append(max(temp2))
        engagement.append(max(temp3))
        valence.append(max(temp4))
        
    #compute the average rPPG during the relevant times
    end_time = max(affect_df["time_stamp"])
    secs_per_bin = end_time / 240
    for i in range(4):
        #rPPG data is always split into 240 equal time bins
        time1 = vid_times[i][0]/secs_per_bin
        time2 = vid_times[i][1]/secs_per_bin
        rows = rppg_df['timebin'].between(time1, time2)
        temp1 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(rppg_df.at[j,'hb-smooth'])
        #rppg.append(stats.mean(temp1))
        rppg.append(max(temp1))

    #pull in the text sentiment polarity and subjectivity.  Values are -1 to 1, so *100 to compare with facial affect.
    sentiment = sentiment_df[sentiment_df["userID"] == userID]
    vid1_pol = sentiment.at[sentiment.index[0],'vid1_pol']*100
    vid2_pol = sentiment.at[sentiment.index[0],'vid2_pol']*100
    vid3_pol = sentiment.at[sentiment.index[0],'vid3_pol']*100
    vid4_pol = sentiment.at[sentiment.index[0],'vid4_pol']*100
    vid1_subj = sentiment.at[sentiment.index[0],'vid1_subj']*100
    vid2_subj = sentiment.at[sentiment.index[0],'vid2_subj']*100
    vid3_subj = sentiment.at[sentiment.index[0],'vid3_subj']*100
    vid4_subj = sentiment.at[sentiment.index[0],'vid4_subj']*100
    polarity_init = [vid1_pol,vid2_pol,vid3_pol,vid4_pol]
    subjectivity_init = [vid1_subj,vid2_subj,vid3_subj,vid4_subj]
    
    #reorder text sentiment based on video order to line up with the rest of the analysis
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    polarity = [0,0,0,0]
    subjectivity = [0,0,0,0]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            polarity[0] = polarity_init[i]
            subjectivity[0] = subjectivity_init[i]
        elif vid_order[i] == '1':
            polarity[1] = polarity_init[i]
            subjectivity[1] = subjectivity_init[i]
        elif vid_order[i] == '2':
            polarity[2] = polarity_init[i]
            subjectivity[2] = subjectivity_init[i]
        elif vid_order[i] == '3':
            polarity[3] = polarity_init[i]
            subjectivity[3] = subjectivity_init[i]
        
    #Pull in user affiliation
    aff = retreive_affiliation(index)
    party = aff[1]
    ideology = aff[2]
    candidate = aff[3]
    like_joe = aff[4]
    like_trump = aff[5]
    
    #apply data to output variables based affiliation
    disgust_dem = disgust[3]
    disgust_demi_repp = disgust[2]
    disgust_repi_demp = disgust[1]
    disgust_rep = disgust[0]
    joy_dem = joy[3]
    joy_demi_repp = joy[2]
    joy_repi_demp = joy[1]
    joy_rep = joy[0]
    engagement_dem = engagement[3]
    engagement_demi_repp = engagement[2]
    engagement_repi_demp = engagement[1]
    engagement_rep = engagement[0]
    valence_dem = valence[3]
    valence_demi_repp = valence[2]
    valence_repi_demp = valence[1]
    valence_rep = valence[0]
    polarity_dem = polarity[3]
    polarity_demi_repp = polarity[2]
    polarity_repi_demp = polarity[1]
    polarity_rep = polarity[0]
    subjectivity_dem = subjectivity[3]
    subjectivity_demi_repp = subjectivity[2]
    subjectivity_repi_demp = subjectivity[1]
    subjectivity_rep = subjectivity[0]
    rppg_dem = rppg[3]
    rppg_demi_repp = rppg[2]
    rppg_repi_demp = rppg[1]
    rppg_rep = rppg[0]

    return [[userID,party,ideology,candidate,like_joe,
              like_trump,'democrat',disgust_dem,joy_dem,engagement_dem,valence_dem,polarity_dem,subjectivity_dem,rppg_dem],
            [userID,party,ideology,candidate,like_joe,
              like_trump,'demi_repp',disgust_demi_repp,joy_demi_repp,engagement_demi_repp,valence_demi_repp,polarity_demi_repp,subjectivity_demi_repp,rppg_demi_repp],
            [userID,party,ideology,candidate,like_joe,
              like_trump,'repi_demp',disgust_repi_demp,joy_repi_demp,engagement_repi_demp,valence_repi_demp,polarity_repi_demp,subjectivity_repi_demp,rppg_repi_demp],
            [userID,party,ideology,candidate,like_joe,
              like_trump,'republican',disgust_rep,joy_rep,engagement_rep,valence_rep,polarity_rep,subjectivity_rep,rppg_rep]]


In [13]:
print(compute_summary_stats(0))

j7WOU
[['j7WOU', 'Republican', 'Right', 'Trump', 3, 4, 'democrat', 43.19906234741211, 90.0262451171875, 98.6795883178711, 84.71695709228516, 20.0, 20.0, 23.354], ['j7WOU', 'Republican', 'Right', 'Trump', 3, 4, 'demi_repp', 90.28540802001952, 0.005586771294474603, 99.83033752441406, 12.989463806152346, 60.0, 90.0, 29.647], ['j7WOU', 'Republican', 'Right', 'Trump', 3, 4, 'repi_demp', 46.09028244018555, 0.005586979445070028, 97.5625228881836, 3.2453601360321045, 60.0, 87.5, 16.962], ['j7WOU', 'Republican', 'Right', 'Trump', 3, 4, 'republican', 63.84717178344727, 0.010058347135782242, 99.91178131103516, 29.245098114013672, 0.0, 0.0, 31.328000000000003]]


In [14]:
#create the summary stats output dataframe

temp_data = [['userID','party','ideology','candidate','like_joe',
              'like_trump','condition','disgust','joy',
              'engagement','valence','polarity','subjectivity','rppg']]
output_df = pd.DataFrame(temp_data, columns = ['userID','party','ideology','candidate','like_joe',
              'like_trump','condition','disgust','joy',
              'engagement','valence','polarity','subjectivity','rppg'])

k = 0
for i in range(num_subjs):
    stats_list = compute_summary_stats(i)
    for j in range(4):
        foo = [stats_list[j]]
        temp_df = pd.DataFrame(foo, columns = ['userID','party','ideology','candidate','like_joe',
                  'like_trump','condition','disgust','joy',
                  'engagement','valence','polarity','subjectivity','rppg'])
        output_df.loc[k] = temp_df.loc[0]
        k += 1


j7WOU
GFPux
mOhA6
Ur76R
2clJY
VRQJ7
lCWiX
5yMwL
6aS64
mTe10
1XVI6
yFcTy
zFHlS
WrJ4w
Ssd0G
qWg6e
Crzgd
LxPDL
LAC2O
PPckh
yavQR
20EGr
RWgXF
dSS3V


In [15]:
output_df.to_csv(r'regression_data_clean_max.csv', index = False)

In [ ]:
#Functions with reverse timestamp indexing

In [19]:
#Get all of the video timestimes and put them in the same order
def retreive_vid_times(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    time_array = times.at[times.index[0],"finalsurvey1time"].split()[0].split(':')
    end_time = 3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])
    
    vid_times = [[0,0],[0,0],[0,0],[0,0]]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[0][0] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[0][1] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
        elif vid_order[i] == '1':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[1][0] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[1][1] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
        elif vid_order[i] == '2':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[2][0] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[2][1] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
        elif vid_order[i] == '3':
            time_array = times.at[times.index[0],"vid{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[3][0] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))
            time_array = times.at[times.index[0],"pc{x}time".format(x=(i+1))].split()[0].split(':')
            vid_times[3][1] = clocks_are_hard(end_time - (3600*float(time_array[0]) + 60*float(time_array[1]) + float(time_array[2])))

    return vid_times
    #!!! Units of time are "seconds from the end"

In [29]:
#compute summary statistics for each video for each participant
def compute_summary_stats(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    vid_times = compiled_vid_times[index]
    
    for r in rppg_data:
        if r[0] == userID:
            rppg_df = r[1]
            break
            
    disgust = []
    joy = []
    engagement = []
    valence = []
    rppg = []
    
    print(userID)  #progress tracker

    #compute the average affect in each facial affect category during the relevant times
    end_time = max(affect_df["time_stamp"])
    for i in range(4):
        #need to convert from "seconds from the end" to time_stamp
        time1 = end_time - vid_times[i][0]
        time2 = end_time - vid_times[i][1]
        rows = affect_df['time_stamp'].between(time1, time2)
        temp1 = []
        temp2 = []
        temp3 = []
        temp4 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(affect_df.at[j,'disgust'])
                temp2.append(affect_df.at[j,'joy'])
                temp3.append(affect_df.at[j,'engagement'])
                temp4.append(affect_df.at[j,'valence'])
        disgust.append(stats.mean(temp1))
        joy.append(stats.mean(temp2))
        engagement.append(stats.mean(temp3))
        valence.append(stats.mean(temp4))
        
    #compute the average rPPG during the relevant times
    secs_per_bin = end_time / 240
    for i in range(4):
        #need to convert from "seconds from the end" to time_stamp
        #rPPG data is always split into 240 equal time bins
        time1 = 240 - (vid_times[i][0]/secs_per_bin)
        time2 = 240 - (vid_times[i][1]/secs_per_bin)
        rows = rppg_df['timebin'].between(time1, time2)
        temp1 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(rppg_df.at[j,'hb-smooth'])
        rppg.append(stats.mean(temp1))

    #pull in the text sentiment polarity and subjectivity.  Values are -1 to 1, so *100 to compare with facial affect.
    sentiment = sentiment_df[sentiment_df["userID"] == userID]
    vid1_pol = sentiment.at[sentiment.index[0],'vid1_pol']*100
    vid2_pol = sentiment.at[sentiment.index[0],'vid2_pol']*100
    vid3_pol = sentiment.at[sentiment.index[0],'vid3_pol']*100
    vid4_pol = sentiment.at[sentiment.index[0],'vid4_pol']*100
    vid1_subj = sentiment.at[sentiment.index[0],'vid1_subj']*100
    vid2_subj = sentiment.at[sentiment.index[0],'vid2_subj']*100
    vid3_subj = sentiment.at[sentiment.index[0],'vid3_subj']*100
    vid4_subj = sentiment.at[sentiment.index[0],'vid4_subj']*100
    polarity_init = [vid1_pol,vid2_pol,vid3_pol,vid4_pol]
    subjectivity_init = [vid1_subj,vid2_subj,vid3_subj,vid4_subj]
    
    #reorder text sentiment based on video order to line up with the rest of the analysis
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    polarity = [0,0,0,0]
    subjectivity = [0,0,0,0]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            polarity[0] = polarity_init[i]
            subjectivity[0] = subjectivity_init[i]
        elif vid_order[i] == '1':
            polarity[1] = polarity_init[i]
            subjectivity[1] = subjectivity_init[i]
        elif vid_order[i] == '2':
            polarity[2] = polarity_init[i]
            subjectivity[2] = subjectivity_init[i]
        elif vid_order[i] == '3':
            polarity[3] = polarity_init[i]
            subjectivity[3] = subjectivity_init[i]
        
    #Pull in user affiliation
    aff = retreive_affiliation(index)
    party = aff[1]
    ideology = aff[2]
    candidate = aff[3]
    like_joe = aff[4]
    like_trump = aff[5]
    
    #apply data to output variables based affiliation
    disgust_dem = disgust[3]
    disgust_demi_repp = disgust[2]
    disgust_repi_demp = disgust[1]
    disgust_rep = disgust[0]
    joy_dem = joy[3]
    joy_demi_repp = joy[2]
    joy_repi_demp = joy[1]
    joy_rep = joy[0]
    engagement_dem = engagement[3]
    engagement_demi_repp = engagement[2]
    engagement_repi_demp = engagement[1]
    engagement_rep = engagement[0]
    valence_dem = valence[3]
    valence_demi_repp = valence[2]
    valence_repi_demp = valence[1]
    valence_rep = valence[0]
    polarity_dem = polarity[3]
    polarity_demi_repp = polarity[2]
    polarity_repi_demp = polarity[1]
    polarity_rep = polarity[0]
    subjectivity_dem = subjectivity[3]
    subjectivity_demi_repp = subjectivity[2]
    subjectivity_repi_demp = subjectivity[1]
    subjectivity_rep = subjectivity[0]
    rppg_dem = rppg[3]
    rppg_demi_repp = rppg[2]
    rppg_repi_demp = rppg[1]
    rppg_rep = rppg[0]

    return [[userID,party,ideology,candidate,like_joe,
              like_trump,'democrat',disgust_dem,joy_dem,engagement_dem,valence_dem,polarity_dem,subjectivity_dem,rppg_dem],
            [userID,party,ideology,candidate,like_joe,
              like_trump,'demi_repp',disgust_demi_repp,joy_demi_repp,engagement_demi_repp,valence_demi_repp,polarity_demi_repp,subjectivity_demi_repp,rppg_demi_repp],
            [userID,party,ideology,candidate,like_joe,
              like_trump,'repi_demp',disgust_repi_demp,joy_repi_demp,engagement_repi_demp,valence_repi_demp,polarity_repi_demp,subjectivity_repi_demp,rppg_repi_demp],
            [userID,party,ideology,candidate,like_joe,
              like_trump,'republican',disgust_rep,joy_rep,engagement_rep,valence_rep,polarity_rep,subjectivity_rep,rppg_rep]]


In [ ]:
###functions that generate wide dataset (one row per participant)

In [13]:
#compute summary statistics for each video for each participant
def compute_summary_stats(index):
    user = affect_data[index]
    userID = user[0]
    affect_df = user[1]
    vid_times = compiled_vid_times[index]
    
    for r in rppg_data:
        if r[0] == userID:
            rppg_df = r[1]
            break
            
    disgust = []
    joy = []
    engagement = []
    valence = []
    rppg = []
    
    print(userID)  #progress tracker

    #compute the average affect in each facial affect category during the relevant times
    for i in range(4):
        time1 = vid_times[i][0]
        time2 = vid_times[i][1]
        rows = affect_df['time_stamp'].between(time1, time2)
        temp1 = []
        temp2 = []
        temp3 = []
        temp4 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(affect_df.at[j,'disgust'])
                temp2.append(affect_df.at[j,'joy'])
                temp3.append(affect_df.at[j,'engagement'])
                temp4.append(affect_df.at[j,'valence'])
        disgust.append(stats.mean(temp1))
        joy.append(stats.mean(temp2))
        engagement.append(stats.mean(temp3))
        valence.append(stats.mean(temp4))
        
    #compute the average rPPG during the relevant times
    end_time = max(affect_df["time_stamp"])
    secs_per_bin = end_time / 240
    for i in range(4):
        #rPPG data is always split into 240 equal time bins
        time1 = vid_times[i][0]/secs_per_bin
        time2 = vid_times[i][1]/secs_per_bin
        rows = rppg_df['timebin'].between(time1, time2)
        temp1 = []
        for j in rows.index:
            if rows[j]:
                temp1.append(rppg_df.at[j,'hb-smooth'])
        rppg.append(stats.mean(temp1))

    #pull in the text sentiment polarity and subjectivity.  Values are -1 to 1, so *100 to compare with facial affect.
    sentiment = sentiment_df[sentiment_df["userID"] == userID]
    vid1_pol = sentiment.at[sentiment.index[0],'vid1_pol']*100
    vid2_pol = sentiment.at[sentiment.index[0],'vid2_pol']*100
    vid3_pol = sentiment.at[sentiment.index[0],'vid3_pol']*100
    vid4_pol = sentiment.at[sentiment.index[0],'vid4_pol']*100
    vid1_subj = sentiment.at[sentiment.index[0],'vid1_subj']*100
    vid2_subj = sentiment.at[sentiment.index[0],'vid2_subj']*100
    vid3_subj = sentiment.at[sentiment.index[0],'vid3_subj']*100
    vid4_subj = sentiment.at[sentiment.index[0],'vid4_subj']*100
    polarity_init = [vid1_pol,vid2_pol,vid3_pol,vid4_pol]
    subjectivity_init = [vid1_subj,vid2_subj,vid3_subj,vid4_subj]
    
    #reorder text sentiment based on video order to line up with the rest of the analysis
    times = df_timestamps[df_timestamps["userID"] == userID]
    vid_order = times.at[times.index[0],'vidOrder'].split(',')
    polarity = [0,0,0,0]
    subjectivity = [0,0,0,0]
    for i in range(len(vid_order)):
        if vid_order[i] == '0':
            polarity[0] = polarity_init[i]
            subjectivity[0] = subjectivity_init[i]
        elif vid_order[i] == '1':
            polarity[1] = polarity_init[i]
            subjectivity[1] = subjectivity_init[i]
        elif vid_order[i] == '2':
            polarity[2] = polarity_init[i]
            subjectivity[2] = subjectivity_init[i]
        elif vid_order[i] == '3':
            polarity[3] = polarity_init[i]
            subjectivity[3] = subjectivity_init[i]
        
    #Pull in user affiliation
    aff = retreive_affiliation(index)
    party = aff[1]
    ideology = aff[2]
    candidate = aff[3]
    like_joe = aff[4]
    like_trump = aff[5]
    
    #apply data to output variables based affiliation
    disgust_dem = disgust[3]
    disgust_demi_repp = disgust[2]
    disgust_repi_demp = disgust[1]
    disgust_rep = disgust[0]
    joy_dem = joy[3]
    joy_demi_repp = joy[2]
    joy_repi_demp = joy[1]
    joy_rep = joy[0]
    engagement_dem = engagement[3]
    engagement_demi_repp = engagement[2]
    engagement_repi_demp = engagement[1]
    engagement_rep = engagement[0]
    valence_dem = valence[3]
    valence_demi_repp = valence[2]
    valence_repi_demp = valence[1]
    valence_rep = valence[0]
    polarity_dem = polarity[3]
    polarity_demi_repp = polarity[2]
    polarity_repi_demp = polarity[1]
    polarity_rep = polarity[0]
    subjectivity_dem = subjectivity[3]
    subjectivity_demi_repp = subjectivity[2]
    subjectivity_repi_demp = subjectivity[1]
    subjectivity_rep = subjectivity[0]
    rppg_dem = rppg[3]
    rppg_demi_repp = rppg[2]
    rppg_repi_demp = rppg[1]
    rppg_rep = rppg[0]

    return [userID,party,ideology,candidate,like_joe,
              like_trump,disgust_dem,joy_dem,engagement_dem,valence_dem,polarity_dem,subjectivity_dem,rppg_dem,
            disgust_demi_repp,joy_demi_repp,engagement_demi_repp,valence_demi_repp,polarity_demi_repp,subjectivity_demi_repp,rppg_demi_repp,
            disgust_repi_demp,joy_repi_demp,engagement_repi_demp,valence_repi_demp,polarity_repi_demp,subjectivity_repi_demp,rppg_repi_demp,
            disgust_rep,joy_rep,engagement_rep,valence_rep,polarity_rep,subjectivity_rep,rppg_rep]


In [14]:
#create the summary stats output dataframe

temp_data = [['userID','party','ideology','candidate','like_joe',
              'like_trump','disgust_dem','joy_dem','engagement_dem','valence_dem','polarity_dem','subjectivity_dem','rppg_dem',
            'disgust_demi_repp','joy_demi_repp','engagement_demi_repp','valence_demi_repp','polarity_demi_repp','subjectivity_demi_repp','rppg_demi_repp',
            'disgust_repi_demp','joy_repi_demp','engagement_repi_demp','valence_repi_demp','polarity_repi_demp','subjectivity_repi_demp','rppg_repi_demp',
            'disgust_rep','joy_rep','engagement_rep','valence_rep','polarity_rep','subjectivity_rep','rppg_rep']]
output_df = pd.DataFrame(temp_data, columns = ['userID','party','ideology','candidate','like_joe',
              'like_trump','disgust_dem','joy_dem','engagement_dem','valence_dem','polarity_dem','subjectivity_dem','rppg_dem',
            'disgust_demi_repp','joy_demi_repp','engagement_demi_repp','valence_demi_repp','polarity_demi_repp','subjectivity_demi_repp','rppg_demi_repp',
            'disgust_repi_demp','joy_repi_demp','engagement_repi_demp','valence_repi_demp','polarity_repi_demp','subjectivity_repi_demp','rppg_repi_demp',
            'disgust_rep','joy_rep','engagement_rep','valence_rep','polarity_rep','subjectivity_rep','rppg_rep'])

k = 0
for i in range(num_subjs):
    foo = [compute_summary_stats(i)]
    temp_df = pd.DataFrame(foo, columns = ['userID','party','ideology','candidate','like_joe',
          'like_trump','disgust_dem','joy_dem','engagement_dem','valence_dem','polarity_dem','subjectivity_dem','rppg_dem',
        'disgust_demi_repp','joy_demi_repp','engagement_demi_repp','valence_demi_repp','polarity_demi_repp','subjectivity_demi_repp','rppg_demi_repp',
        'disgust_repi_demp','joy_repi_demp','engagement_repi_demp','valence_repi_demp','polarity_repi_demp','subjectivity_repi_demp','rppg_repi_demp',
        'disgust_rep','joy_rep','engagement_rep','valence_rep','polarity_rep','subjectivity_rep','rppg_rep'])
    output_df.loc[k] = temp_df.loc[0]
    k += 1


j7WOU
GFPux
mOhA6
Ur76R
2clJY
VRQJ7
lCWiX
5yMwL
6aS64
mTe10
1XVI6
yFcTy
zFHlS
WrJ4w
Ssd0G
qWg6e
Crzgd
LxPDL
LAC2O
PPckh
yavQR
20EGr
RWgXF
dSS3V
